In [1]:
import numpy as np
import pandas as pd
from bokeh.charts import Scatter
from bokeh.plotting import Figure
from bokeh.models import Span
from bokeh.io import output_notebook, show
output_notebook()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


Loading BokehJS ...

# Linear Regression

As discussed in the videos, one part of both statistics and machine learning is linear regression.  There are several Python libraries for this as well.  The one we'll be using in this course is the **scikit-learn [LinearRegression class](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)**, which uses the **ordinary least squares** method to calculate predictions.

We will perform linear regression on one of the sample data sets shipped with Bokeh: the "Iris" dataset, which consists of data about flowers.  First, we import and make a copy of the `flowers` dataset, and display the fields associated with it to get an idea of the data:

In [2]:
# Creating a DataFrame using the sample data provided by Bokeh
from bokeh.sampledata.iris import flowers
flowers_data = flowers

# Displaying first few rows of data
flowers_data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


We are going to attempt to see **what features are best used** to calculate the [sepal](https://en.wikipedia.org/wiki/Sepal).  We start by creating an instance of the `LinearRegression` class

In [3]:
# lm is an instance of the LinearRegression class
lm = LinearRegression()

## Fitting the Model

We are going to begin by creating a model using the petal length (as the **predictor variable**) to calculate the sepal length (the **response** variable).

We need some X and y values, which will be used to call the **`fit`** function.  Create a variable X for predictor variables, and a variable y for the associated response variable.  These will be data frames taken from the `flowers_data` variable.

In [4]:
X = flowers_data[['petal_length']]
y = flowers_data['sepal_length']

We now use the Bokeh `Figure` class to illustrate the distribution of this predictor:

Follow this link to find out more about [Bokeh.plotting](http://bokeh.pydata.org/en/0.11.0/docs/reference/plotting.html).

In [5]:
fig = Figure()
fig.circle(X['petal_length'],y)

show(fig)

Having prepared the data, we can now fit the data to a linear regression model, and then begin analysis:

In [6]:
# Call the fit function on the data
lm = LinearRegression()
lm.fit(X, y)
print(vars(lm))

{'copy_X': True, 'normalize': False, '_residues': 24.525033765831754, 'fit_intercept': True, 'coef_': array([ 0.40892228]), 'singular_': array([ 21.54821106]), 'n_jobs': 1, 'rank_': 1, 'intercept_': 4.3066034150475794}


Having fitted the model to the data, we can establish what the regression line by using the `coef_` and `intercept_` values, which we can use in order to generate an equation in the format `y = mx + c`:

In [7]:
m = lm.coef_[0]
c = lm.intercept_
print('y = %fx + %f' % (m,c))


y = 0.408922x + 4.306603


We don't necessarily need to use the equation, however, there is additionally a useful **`predict`** function which allows us to enter one or more numbers for which it will predict the outcome.

In [8]:
var = 0

print(lm.predict([var]))

# Note that like with the earlier Series, the `predict` function is prepared for the fact that each value
# to predict could have multiple features.  So it accepts a list of lists
# This will give a deprecation warning
var = [[3], [5], [4]]
lm.predict(var)

# if a warning message box appears this indicates a possible depracated code issue, don't worry - 
# you have not broken the internet!

[ 4.30660342]


/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 5.53337025,  6.3512148 ,  5.94229252])

# Evaluation of a Model
## Training and Testing Data
In order to ensure that our model can cope with the whole of the dataset, we will split our data into a **training set** and a **testing set**, and **cross validate** the results.  We need to be careful that the ordering of our data is sufficiently **random** when doing this.  For example, if it was sorted by value, we would be training only on a specific type of value making our model less accurate against values outside that range.

The function **['train_test_split'](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)** from the `sklearn.model_selection` module performs this task for us - both splitting the data, and ensuring that the ordering is sufficiently random.  It returns four values, we will assign each of those a value as the output of this function by declaring them on the same line as follows:

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Residual Analysis

In order to ensure that our model is appropriate for the data, we first make an analysis of the residuals, i.e., the difference between the predicted value and the observed value.  If the data are randomly clustered around 0, then that is a sign the data are appropriate.  However, if there is a pattern in the data, then that suggests that there is some form of bias.

We can use the `predict` method of the `LinearRegression` class to get the predicted values (which we'll use as the  `X` values), and we subtract the actual values from them, as follows:

In [10]:
pred_values = lm.predict(X)
pred_difference = (pred_values - y)
fig = Figure()
fig.circle(pred_values,pred_difference)
fig.line(x=[pred_values.min(), pred_values.max()], y=[0,0])
show(fig)

## $R^2$

The $R^2$ value provides us with the **proportion of variance** which is explained by the model.  It is a value between 0 and 1, with 0 being that the model has no explanation for any of the variance and 1 being that the regression line fits the data perfectly.

The defined range of $R^2$ means that it can give a reasonable indication of how good a model is, although there is no defined range of what constitutes a good enough score.  This would naturally vary between domains, where there is a smaller margin of error than others.

In scikit-learn, the **`score`** method calculates the $R^2$ value as follows:

In [11]:
lm.score(X, y)

0.75995464577251504

## RMSE

We now use **RMSE** (Root Mean Squared Error) to **evaluate** our model.  Whereas the residual analysis considers the individual residual difference, RMSE takes the mean of these residuals squared and takes the square root of the total.  This output is the **standard deviation** of a model, and can be used as a measure of accuracy.

Further information about [mean_squared_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) and [square root](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sqrt.html)

There is a function for calculating the mean squared error, and then a numpy function to calculate the square root of this.  We test the ***actual*** data against the ***predicted*** data performed in the previous cell as follows:

In [12]:
np.sqrt(mean_squared_error(X, y))

2.365079279855117

By default, the `test_size` argument is 0.25, which means that when it comes to validating the model, we test on a quarter of the data, having trained our model on the complement (three quarters).  Note the difference in size between the training and testing set, and the ordering of the indexes:

In [13]:
print('Training data: %d\n' % X_train.size, X_train.head(), '\nTest data: %d\n' % X_test.size, X_test.head())

Training data: 120
      petal_length
20            1.7
103           5.6
107           6.3
51            4.5
143           5.9 
Test data: 30
      petal_length
114           5.1
130           6.1
112           5.5
119           5.0
46            1.6


Our model gets trained as follows:

In [14]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
print(y_pred)

[ 6.37741222  6.78281927  6.53957504  6.33687151  4.95848754  6.29633081
  5.93146446  6.62065645  6.45849363  5.08010965  6.78281927  6.2152494
  6.13416799  6.78281927  6.29633081  4.91794683  4.83686542  6.29633081
  4.99902824  5.93146446  5.97200517  5.80984235  6.62065645  6.74227856
  4.95848754  6.41795292  4.87740613  5.08010965  4.83686542  6.01254587]


## Try it yourself...

There are two more features which you can use to predict the sepal length of a flower.  Try and find the best combination of predictor variables in the cells below.  If you need to create a new cell, click on `Insert -> Insert Cell Below`

In [15]:
# YOUR CODE HERE


In [16]:
# YOUR CODE HERE


# Classification

To finish, we will walk through an example classification with a spam filter.  The process with scikit-learn is similar to the one described for linear regression.  

We won't go into too much detail on classification, which is the other form of supervised machine learning.  However in this notebook, we will present an example of using a **Bayesian classifier** to classify whether SMS text messages are "spam" or "ham".

We use the dataset from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).  These data are stored in the same directory as the notebook in [spam.csv](./spam.csv)

To start, we import the libraries we require to perform this classification, and load the data into a `DataFrame`.  Note that although it's a CSV file, we specify `sep` value as being `'\t'`, which means that the two fields are separated by a tab character rather than a comma.

In [17]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.naive_bayes as nb
import sklearn.feature_extraction.text as text
import sklearn.model_selection as cv
df = pd.read_csv('spam.csv', sep='\t')
df.head()


,Classification,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In order to be able to get the features, we need to identify features (i.e., the words) which are classified in a particular way.  We need to get some measure of the way that the words are distributed within the messages, and use that to predict the classification into Ham or Spam.

There are several ways of assigning the weight to a particular word.  [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is an information retrieval technique to identify the importance of words in a document based on the times a word is in an individual message, but taking account for the times it appears in the whole document - since some words generally appear more often than others.

The ["bag of words"](https://en.wikipedia.org/wiki/Bag-of-words_model) is simple: it simply ignores grammar and makes a count of the times that a particular word appears.  This is a commonly used approach, which we will use below.

Scikit-learn allows a particular `Vectorizer` (in this instance [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)) to be selected to implement one of these strategies, as follows:

In [18]:
counts = text.CountVectorizer()

For scikit-learn to be able to process the data, it needs the data to be as numbers.  The [fit_transform](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform) function converts these data, and we convert the classification to a boolean, which Python can consider as an integer.

We define the features as `X` and the labels as `y`

In [19]:
# The fit_transform function 
X = counts.fit_transform(df['Message'])


# Changing the classification field to a boolean, Python can regard it as an integer of 1 or 0
y = df['Classification'] == 'spam'
#print(X, y.head())

As with linear regression, we split the data into a training and a testing set using the `train_test_split` function, and we fit the training data to the classifier.

There are several different types of naive bayes classifier we can use.  In this instance, we are using the [multinomial naive bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) classifier:

In [20]:
# Split to a training set and a testing set
(text_train, text_test, label_train, label_test) = cv.train_test_split(X, y, test_size=0.2)

from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(text_train, label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Having trained the model, we can get a simple test of the accuracy by using the [score](http://scikit-learn.org/stable/modules/model_evaluation.html) method:

In [21]:
classifier.score(text_test, label_test)

0.97578475336322867

We can see that the classifier is already performing very well.  Note that it will give a slightly different result each time, because the classifier is trained on different messages as the order is randomised.

The [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) shows us the breakdown of instances of true positives, true negatives, false positives, and false negatives.  There is a method for this as well, used upon the test data.  

In [22]:
from sklearn.metrics import confusion_matrix, f1_score

predictions = classifier.predict(text_test)
confusion_matrix(label_test, predictions)

array([[956,  17],
       [ 10, 132]])

It is possible to try our own messages as well, and see how those perform.  Once again, they need to be transformed into the correct format for scikit-learn.  The output is as `False` (not spam) and `True` (spam):

In [23]:
messages = ["this is a test", "spam, spam, spam, glorious spam"]
transformed_messages = counts.transform(messages)
predictions = classifier.predict(transformed_messages)
print(predictions)

[False  True]
